# Visão Computacional - Encontrando as guias-Parte 4


## Otimizando o Código

In [2]:
import cv2 
import numpy as np


def canny(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    canny = cv2.Canny(blur,50,150)
    return canny

def display_lines(image,lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
            cv2.line(line_image, (x1,y1),(x2,y2),(255,0,0),10)
    return line_image    

def regiao_de_interresse(image):
    altura = image.shape[0]                                           
    triangulo = np.array([[(220,altura), (1100,altura),(550,250)]]) 
    mask = np.zeros_like(image)         
    cv2.fillPoly(mask, triangulo, 255) 
    mask_image = cv2.bitwise_and(image,mask)
    return mask_image

image = cv2.imread('test_image.jpg')
image_estrada = np.copy(image)
canny_image = canny(image_estrada)
cropado = regiao_de_interresse(canny_image)
lines = cv2.HoughLinesP(cropado, 2 ,np.pi/180, 100 ,np.array([]), minLineLength = 40 , maxLineGap = 5)
line_image = display_lines(image_estrada,lines)
imagens_unidas = cv2.addWeighted(image_estrada, 0.8 , line_image , 1 , 1)

Queremos otimizar a linhas azuis que vimos no livro anterior

In [9]:
def fazer_coordenadas(image, parametro_linha):
    inclinacao , intercecao = parametro_linha
    y1 = int(image.shape[0])# bottom of the image
    y2 = int(y1*3/5)         # slightly lower than the middle
    x1 = int((y1 - intercecao)/inclinacao)
    x2 = int((y2 - intercecao)/inclinacao)
    return [[x1, y1, x2, y2]]
 
def media_inclinacao_intercecao(image, lines):
    parte_esquerda    = []
    parte_direita   = []
    if lines is None:
        return None
    for line in lines:
        for x1, y1, x2, y2 in line:
            fit = np.polyfit((x1,x2), (y1,y2), 1)
            inclinacao = fit[0]
            intercecao = fit[1]
            if inclinacao < 0: # y is reversed in image
                parte_esquerda.append((inclinacao, intercecao))
            else:
                parte_direita.append((inclinacao, intercecao))
    
    parte_esquerda_average  = np.average(parte_esquerda, axis=0)
    parte_direita_average = np.average(parte_direita, axis=0)
    left_line  = fazer_coordenadas(image, parte_esquerda_average)
    right_line = fazer_coordenadas(image, parte_direita_average)
    averaged_lines = [left_line, right_line]
    return averaged_lines
 
image = cv2.imread('test_image.jpg')
image_estrada = np.copy(image)
canny_image = canny(image_estrada)
cropado = regiao_de_interresse(canny_image)
lines = cv2.HoughLinesP(cropado, 2 ,np.pi/180, 100 ,np.array([]), minLineLength = 40 , maxLineGap = 5)
line_image = display_lines(image_estrada,lines)
imagens_unidas = cv2.addWeighted(image_estrada, 0.8 , line_image , 1 , 1)
 
cv2.imshow("resultado",imagens_unidas)
cv2.waitKey(0)

-1